# 2 output CNN

In this section, we are going to construct convolutional neural network (CNN) with 2 output.

# 1. Load dependencies

First, we can load tf.keras and other python libraries.

In [7]:
import numpy as np
np.random.seed(2 ** 10)

import matplotlib.pyplot as plt

import tensorflow as tf
print("[INFO] Tensorflow version is {}".format(tf.__version__))

tf.enable_eager_execution()

[INFO] Tensorflow version is 1.11.0


# 2. Load dataset and pre-processing

In [2]:
# Load cifar10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Pre-processing
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255.
x_test /= 255.

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10).astype(np.float32)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10).astype(np.float32)

print("Shape of x_train is {}".format(np.shape(x_train)))
print("Shape of x_test is {}".format(np.shape(x_test)))
print("Shape of y_train is {}".format(np.shape(y_train)))
print("Shape of y_test is {}".format(np.shape(y_test)))

Shape of x_train is (50000, 32, 32, 3)
Shape of x_test is (10000, 32, 32, 3)
Shape of y_train is (50000, 10)
Shape of y_test is (10000, 10)


# 3. Construct 2 output CNN with `tf.keras`.

We design a network for two purposes.

One is for reconstruction purpose, the next one is for classification purpose network.

For these two purposes, we design a shared encoder network, a decoder for reconstruction, and an fc layer for classification.

In [8]:
# Build a tf.keras model

# 1) Encoder layer for feature extraction
inputs = tf.keras.Input(shape=(32, 32, 3))

conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, 
                               kernel_initializer="he_normal",
                               kernel_regularizer=tf.keras.regularizers.l2(5e-4),
                               padding="same", activation=tf.nn.relu)(inputs)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv1)
conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, 
                               kernel_initializer="he_normal",
                               kernel_regularizer=tf.keras.regularizers.l2(5e-4),
                               padding="same", activation=tf.nn.relu)(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv2)

# 2) Decoder layer for reconstruction
trans_conv1 = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3,
                                    kernel_initializer="he_normal",
                                    kernel_regularizer=tf.keras.regularizers.l2(5e-4),
                                    strides=1, padding="same",
                                    activation=tf.nn.relu)(pool2)
unpool1 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(trans_conv1)
trans_conv2 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3,
                                    kernel_initializer="he_normal",
                                    kernel_regularizer=tf.keras.regularizers.l2(5e-4),
                                    strides=1, padding="same",
                                    activation=tf.nn.relu)(unpool1)
unpool2 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format="channels_last")(trans_conv2)
reconstruction = tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3,
                                    kernel_initializer="he_normal",
                                    kernel_regularizer=tf.keras.regularizers.l2(5e-4),
                                    strides=2, padding="same",
                                    activation=tf.nn.sigmoid, name="output_2")(unpool1)

# 3) Fc layer for classification
flatten = tf.keras.layers.Flatten()(pool2)
fc1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flatten)
fc2 = tf.keras.layers.Dense(64, activation=tf.nn.relu)(fc1)
probabilities = tf.keras.layers.Dense(10, activation=tf.nn.softmax, name="output_1")(fc2)

model = tf.keras.Model(inputs=[inputs], outputs=[probabilities, reconstruction])
model.summary()
print("Total parameters of model is {}".format(model.count_params()))  #468984
print("model output is {}".format(model.outputs))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 32, 32, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 16, 16, 32)   0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 16, 16, 64)   18496       max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
max_poolin

In [9]:
opt = tf.train.RMSPropOptimizer(learning_rate=0.0001)
model.compile(loss=['categorical_crossentropy', 'mse'],
              optimizer=opt, loss_weights=[1., 0.05],
              metrics=["accuracy", "mse"])

BATCH_SIZE = 64
SHUFFLE_SIZE = 10000 

# 4. Dataset pipeline

**The following is the most important part of this section.**

Basically we use `tf.data`, but we build custom data pipeline to match the output of the network.

In [13]:
# Create function for dataset pipeline
def _input_fn(x, y):

    # Custom dataset loader for 2 output network
    def generator():
        for images, labels in zip(x, y):
            yield images, {"output_1": labels, "output_2": images}

    dataset = tf.data.Dataset.from_generator(generator, output_types=(tf.float32, {"output_1": tf.float32, "output_2": tf.float32}))
    dataset = dataset.shuffle(SHUFFLE_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

# 5. Train model

Finally, train our model for checking accuracy!!

In [12]:
for epoch in range(10):
    total_loss, prediction_loss, recon_loss, \
    prediction_acc, _, _, recon_error = model.train_on_batch(_input_fn(x_train, y_train))
    _, test_loss, _, test_acc, _, _, _ = model.test_on_batch(_input_fn(x_test, y_test))
    print('Epoch #{}\t Loss: #{}\tAccuracy: #{}'.format(epoch + 1, test_loss, test_acc))

Epoch #1	 Loss: #2.454409599304199	Accuracy: #0.046875
Epoch #2	 Loss: #2.4976654052734375	Accuracy: #0.109375
Epoch #3	 Loss: #2.2861523628234863	Accuracy: #0.171875
Epoch #4	 Loss: #2.3842673301696777	Accuracy: #0.078125
Epoch #5	 Loss: #2.373509645462036	Accuracy: #0.109375
Epoch #6	 Loss: #2.363190174102783	Accuracy: #0.140625
Epoch #7	 Loss: #2.413235902786255	Accuracy: #0.078125
Epoch #8	 Loss: #2.3201699256896973	Accuracy: #0.125
Epoch #9	 Loss: #2.5095200538635254	Accuracy: #0.078125
Epoch #10	 Loss: #2.501054525375366	Accuracy: #0.09375
